## Configuration

In [1]:
! nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [3]:
# imports
import os

import torch, torchvision
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
from torchvision.datasets import CIFAR10

torch.__version__, torchvision.__version__
torch.cuda.is_available()
# Install Albumentation library
%pip install albumentations -qqq
# Install LR finder
%pip install torchinfo torch_lr_finder -qqq
# Install GradCam
%pip install grad-cam -qqq

%pip install tqdm -qqq

%pip install torchscan --no-deps -qqq

%pip install seaborn -qqq

%pip install gradio -qqq

%pip install pytorch-lightning -qqq

%pip install torchmetrics -qqq

%pip install wandb -qqq

import seaborn as sn  # for heatmaps
import math
from collections import OrderedDict
import sys
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR, OneCycleLR, ReduceLROnPlateau
import torchmetrics
from torchmetrics import Accuracy
import matplotlib.pyplot as plt

from torch_lr_finder import LRFinder
import copy
from torchvision.utils import make_grid, save_image

import pytorch_lightning as pl
from pytorch_lightning import LightningDataModule, LightningModule, Trainer
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import LearningRateMonitor, ModelSummary

# Weights & Biases
import wandb
from pytorch_lightning.loggers import WandbLogger

import gradio as gr

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
!git clone https://github.com/nanekja/pytorch_utils -qqq

sys.path.append('./pytorch_utils/')
import main
import utils
import transform
import train1
import test1
import dataloader
import albumentation
import gradcam
import C10_DM_Lit

sys.path.append('./pytorch_utils/models/')
import c_resnet
import Lit_c_res

import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image

fatal: destination path 'pytorch_utils' already exists and is not an empty directory.


/home/nanekja/.local/lib/python3.10/site-packages/albumentations/augmentations/dropout/cutout.py:49: FutureWarning: Cutout has been deprecated. Please use CoarseDropout
  warnings.warn(


Files already downloaded and verified
Files already downloaded and verified


ModuleNotFoundError: No module named 'gradcam'

## DATA

In [ ]:
batch_size = 512
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
EPOCHS=20

In [ ]:
#PATH_DATASETS = os.environ.get("PATH_DATASETS", ".")
#AVAIL_GPUS = min(1, torch.cuda.device_count())
#BATCH_SIZE = 256 if AVAIL_GPUS else 64
#AVAIL_GPUS

In [6]:
max_epochs=EPOCHS

# Create the data module
#data_module = CIFAR10DataModule()
data_module = C10_DM_Lit.CIFAR10DataModule()

#model = Lit_c_resnet()
model = Lit_c_res.Lit_c_resnet()
trainer = Trainer(max_epochs=max_epochs,accelerator="auto")

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
trainer.fit(model,datamodule=data_module)

In [ ]:
trainer.test(model,datamodule=data_module)

In [ ]:
# start tensorboard
%load_ext tensorboard
%tensorboard --logdir lightning_logs/

In [ ]:
torch.save(model.state_dict(), "model.pth")

In [ ]:
mean, std = utils.get_mean_and_std()
device = utils.get_device()

wp = utils.wrong_predictions(model, transform.test_loader, device)
utils.plot_misclassified2(wp, mean, std, 10, classes)

In [ ]:
target_layers = ["layer4"]

gradcam_output, probs, predicted_classes = gradcam.generate_gradcam(wp[:10], model, target_layers, device)
gradcam.plot_gradcam(gradcam_output, target_layers, classes, (3, 32, 32),predicted_classes, wp[:10], mean, std)

In [ ]:
targets = None
# Denormalize the data using test mean and std deviation
inv_normalize = transforms.Normalize(
    mean=[-0.50/0.23, -0.50/0.23, -0.50/0.23],
    std=[1/0.23, 1/0.23, 1/0.23]
)

In [ ]:
import numpy as np
import gradio as gr
from PIL import Image
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image

def inference(input_img, transparency):
    transform = transforms.ToTensor()
    input_img = transform(input_img)
    input_img = input_img.to(device)
    input_img = input_img.unsqueeze(0)
    outputs = model(input_img)
    _, prediction = torch.max(outputs, 1)
    target_layers = [model.layer2[-2]]
    cam = GradCAM(model=model, target_layers=target_layers, use_cuda=True)
    grayscale_cam = cam(input_tensor=input_img, targets=targets)
    grayscale_cam = grayscale_cam[0, :]
    img = input_img.squeeze(0).to('cpu')
    img = inv_normalize(img)
    rgb_img = np.transpose(img, (1, 2, 0))
    rgb_img = rgb_img.numpy()
    visualization = show_cam_on_image(rgb_img, grayscale_cam, use_rgb=True, image_weight=transparency)
    return classes[prediction[0].item()], visualization

demo = gr.Interface(inference, [gr.Image(shape=(32, 32)), gr.Slider(0, 1)], ["text", gr.Image(shape=(32, 32)).style(width=128, height=128)])
demo.launch()